In [1]:
from pathlib import Path
import sqlite3
from helper import load_dotenv
import re
from tqdm import tqdm

In [2]:
config = load_dotenv()

In [3]:
config

{'DATA_DIR': PosixPath('/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data'),
 'GENERIC_DATA_DIR': PosixPath('/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/21_generic_data'),
 'SORT_DIR': PosixPath('/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/20_data/00_to_sort'),
 'DB_PATH': PosixPath('/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/10_db/moire_testdata.db'),
 'LABELING_JSON_PATH': PosixPath('/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/11_annotation_data'),
 'MODEL_DIR': PosixPath('/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/22_andere_daten/models'),
 'PATTERN_DIR': PosixPath('/Users/frederic.birwe/Library/CloudStorage/OneDrive-BertelsmannSE&Co.KGaA/Masterarbeit/22_andere_daten/patterns'),
 'LOFI_DPI': 600,
 'LOFI_SCREEN_DPI': 300,
 'TR

# Clean Up Files

In [ ]:
files = [
    f for f in config['DATA_DIR'].glob('./**/*')
    if f.is_file() and f.name != '.DS_Store' and f.parent.name != 'pdf'
]

In [ ]:
for f in tqdm(files):
    res = re.match(r'(.+\.p\d+)\.p1(.+)', f.name)
    
    if res:
        new_file_name = f.parent / f'{res.groups()[0]}{res.groups()[1]}'
        f.rename( new_file_name )

# Clean Up DB

In [4]:
con = sqlite3.connect( config['DB_PATH'] )

## Clean Up PDF Pages

In [5]:
c = con.cursor()
c.execute('SELECT * FROM pdf_page')
pdf_pages = c.fetchall()
c.close()

In [6]:
unused_rows = []

for p in pdf_pages:
    pdf_path = config['DATA_DIR'] / p[1] / 'pdf' / f'{ p[0] }.pdf'
    
    if pdf_path.exists() == False:
        unused_rows.append(p)

In [8]:
c = con.cursor()
for p in unused_rows:
    c.execute(f'''
        DELETE FROM pdf_page
        WHERE filename="{ p[0] }" AND job="{ p[1] }"
    ''')
    
c.close()
con.commit()

## Clean Up related_file

In [9]:
c = con.cursor()
c.execute('''
    SELECT rf.*, ifnull(pp.pdf_exists, 0) AS pdf_exists FROM related_file rf
    LEFT JOIN (
        SELECT filename, 1 AS pdf_exists 
        FROM pdf_page pp 
    ) pp
    ON rf.pdf_filename = pp.filename 
''')
related_files = c.fetchall()
c.close()

In [10]:
file_not_available = []

for rf in related_files:
    rf_path = config['DATA_DIR'] / str(rf[2]) / rf[0] / rf[4]
    
    if rf_path.exists() == False:
        file_not_available.append(rf)

In [12]:
c = con.cursor()
for rf in file_not_available:
    c.execute(f'''
        DELETE FROM related_file
        WHERE variant_name="{ rf[0] }" AND pdf_filename="{ rf[1] }" AND job="{ rf[2] }"
    ''')
    
c.close()
con.commit()

In [13]:
pdf_not_available = [
    rf for rf in related_files
    if rf[5] == 0
]

In [14]:
c = con.cursor()
for rf in pdf_not_available:
    c.execute(f'''
        DELETE FROM related_file
        WHERE variant_name="{ rf[0] }" AND pdf_filename="{ rf[1] }" AND job="{ rf[2] }"
    ''')
    
c.close()
con.commit()